# EMSRb / EMSRb-MR example

In this example, we demonstrate the calculation of booking limits using both the traditional __EMSRb__ algorithm and the more recent __EMSRb-MR__ algorithm. MR stands for marginal revenue transformation - this transformation (also called "fare transformation") transforms the demands and prices in such a way that traditional optimisers designed for independent demand can also be used in the realm of less restricted fare structures. 

The example data given here corresponds to a fully unrestricted fare structure and is taken from the following reference:

`[1]` Fiig et al: Optimization of Mixed Fare Structures: Theory and Applications _Journal of Revenue and Pricing Management_, 9, 1, 152–170 (2010)

As of December 2016, this article was publicly available here: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.474.1315&rep=rep1&type=pdf

In [1]:
import sys

import numpy as np
import pandas as pd

sys.path.append('../')
from pyrm.pyrm import protection_levels
from pyrm.helpers import cumulative_booking_limits
from pyrm.fare_transformation import calc_fare_transformation

## 1) Define example data

In [2]:
fares = np.array([1200, 1000, 800, 600, 400, 200])
demands = np.array([31.2, 10.9, 14.8, 19.9, 26.9, 36.3])
sigmas = np.array([11.2, 6.6, 7.7, 8.9, 10.4, 12])
capacity = 100
classes = np.arange(len(fares)) + 1
class_names = ["class_{}".format(c) for c in classes]

## 2) Carry out the fare transformation

In [18]:
adjusted_fares, adjusted_demand, Q, TR = calc_fare_transformation(fares, demands, return_all=True)

## 3) Reproduce table 1 from ref. `[1]`
Note: 
- here, `Q` and `TR` are only reproduced for the fully undifferentiated case
- classes with negative adjusted fare have a NaN in `Q` and `TR`, since inefficient fares are removed during the fare transformation routine
- some values slightly differ from the values shown in ref. `[1]`, probably due to rounding errors

In [19]:
data = np.transpose(np.vstack((fares, demands, Q, TR, adjusted_fares)))
pd.DataFrame(data, index=class_names, columns=['fares', 'demand', 'Q', 
                                               'TR', 'MR/adjusted fares'])

,fares,demand,Q,TR,MR/adjusted fares
class_1,1200.0,31.2,31.2,37440.0,1200.000000
class_2,1000.0,10.9,42.1,42100.0,427.522936
class_3,800.0,14.8,56.9,45520.0,231.081081
class_4,600.0,19.9,76.8,46080.0,28.140704
class_5,400.0,26.9,NaN,NaN,NaN
class_6,200.0,36.3,NaN,NaN,NaN


## 4a) Calculate EMSRb bookings limits

In [20]:
p = protection_levels(fares, demands, sigmas, method='EMSRb')
booking_limits_EMSRb = cumulative_booking_limits(p, capacity)

print(booking_limits_EMSRb)

[ 100.   80.   65.   46.   20.    0.]


## 4b) Calculate EMSRb-MR booking limits

In [21]:
p = protection_levels(fares, demands, sigmas, method='EMSRb_MR')
booking_limits_EMSRb_MR = cumulative_booking_limits(p, capacity)

print(booking_limits_EMSRb_MR)

[ 100.   65.   48.   16.   nan   nan]


## 5) Reproduce table 2 from  ref. `[1]`

In [23]:
data = np.transpose(np.vstack((fares, demands, sigmas, booking_limits_EMSRb, 
                               adjusted_fares, booking_limits_EMSRb_MR)))
pd.DataFrame(data, index=class_names, columns=['fares', 'demand', 'standard deviation', 
                                               'EMSRb limits', 'adjusted fares', 
                                               'EMSRb-MR limits'])

,fares,demand,standard deviation,EMSRb limits,adjusted fares,EMSRb-MR limits
class_1,1200.0,31.2,11.2,100.0,1200.000000,100.0
class_2,1000.0,10.9,6.6,80.0,427.522936,65.0
class_3,800.0,14.8,7.7,65.0,231.081081,48.0
class_4,600.0,19.9,8.9,46.0,28.140704,16.0
class_5,400.0,26.9,10.4,20.0,NaN,NaN
class_6,200.0,36.3,12.0,0.0,NaN,NaN
